In [ ]:
import logging
import math
import random
from collections import defaultdict
from functools import partial

import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import open3d as o3d
import torch
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.renderer import (
    CamerasBase,
    FoVPerspectiveCameras,
    MeshRasterizer,
    RasterizationSettings,
    look_at_view_transform,
)
from pytorch3d.structures import Meshes
from neural_poisson.data.prepare import load_mesh, extract_surface_data

model_idx = 9
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCore/04256520"
model_id = sorted(list(Path(shapenet_dir).iterdir()))[model_idx].stem
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

R, T = look_at_view_transform(1.0, 10, 180)
camera = FoVPerspectiveCameras(device="cuda", R=R, T=T, fov=60.0)

mesh = load_mesh(shapenet_path, "cuda")

data = extract_surface_data(
    camera=camera,
    mesh=mesh,
    image_size=512,
    fill_depth="max2",
)

normals = (data["normal_map"] + 1) / 2
plt.imshow(normals.detach().cpu().numpy())

In [1]:
model_id

NameError: name 'model_id' is not defined

In [ ]:
from neural_poisson.mesh.utils import pytorch3d_to_open3d
import open3d as o3d

_mesh = pytorch3d_to_open3d(mesh)
o3d.visualization.draw_plotly([_mesh])

In [ ]:
list(Path(shapenet_dir).iterdir())[0]

In [ ]:
normals.min()

In [ ]:
 list(Path(shapenet_dir).iterdir())

In [ ]:
list(shapenet_dir.iterdir())